In [143]:
from pyod.models.knn import KNN
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math

In [144]:
#44009 and 44065 were the two buoys lying right on the trajectory of the hurricane 
#so we decided to use them to analyse the data further
df_b7_init = pd.read_csv('/Users/shreyabanerjee/DA_proj/clean_csvs/44009_clean.csv')
df_b8_init = pd.read_csv('/Users/shreyabanerjee/DA_proj/clean_csvs/44065_clean.csv')

#adding the two dataframes
df=pd.concat([df_b7_init,df_b8_init],axis=0)

In [145]:
keep_col=['pressure']#,'waveheight','windspeed']
print(df[keep_col].head())
x=df[keep_col].values
y=df['possibility'].values


   pressure
0    1022.8
1    1023.3
2    1023.4
3    1023.0
4    1022.7


In [146]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [147]:
def estimateGaussian(X):
    n = np.size(X, 1)
    m = np.size(X, 0)
    mu = np.zeros((n, 1))
    sigma2 = np.zeros((n, 1))
    
    mu = np.reshape((1/m)*np.sum(X, 0), (1, n))
    sigma2 = np.reshape((1/m)*np.sum(np.power((X - mu),2), 0),(1, n))
    
    return mu, sigma2

mu, sigma2 = estimateGaussian(X_train)

In [148]:
print(mu,sigma2)

[[1015.58441123]] [[66.02676066]]


In [149]:

def multivariateGaussian(X, mu, sigma2):
     n = np.size(sigma2, 1)
     m = np.size(sigma2, 0)
     #print(m,n)
     
     if n == 1 or m == 1:
        # print('Yes!')
         sigma2 = np.diag(sigma2[0, :])
     #print(sigma2)
     X = X - mu
     pi = math.pi
     det = np.linalg.det(sigma2)
     inv = np.linalg.inv(sigma2)
     val = np.reshape((-0.5)*np.sum(np.multiply((X@inv),X), 1),(np.size(X, 0), 1))
     #print(val.shape)
     p = np.power(2*pi, -n/2)*np.power(det, -0.5)*np.exp(val)
     
     return p

In [150]:
p = multivariateGaussian(X_train, mu, sigma2)
print(p.shape)

(11655, 1)


In [151]:
pval = multivariateGaussian(X_test, mu, sigma2)
#pval.shape
y_test.shape

(4996,)

In [152]:
def selectThreshHold(yval, pval):
    
    F1 = 0
    bestF1 = 0
    bestEpsilon = 0
    
    stepsize = (np.max(pval) - np.min(pval))/1000
        
    epsVec = np.arange(np.min(pval), np.max(pval), stepsize)
    noe = len(epsVec)
    
    for eps in range(noe):
        epsilon = epsVec[eps]
        pred = (pval < epsilon)
        prec, rec = 0,0
        tp,fp,fn = 0,0,0
        
        try:
            for i in range(np.size(pval,0)):
                if pred[i] == 1 and yval[i] == 1:
                    tp+=1
                elif pred[i] == 1 and yval[i] == 0:
                    fp+=1
                elif pred[i] == 0 and yval[i] == 1:
                   fn+=1
            prec = tp/(tp + fp)
            rec = tp/(tp + fn)
            F1 = 2*prec*rec/(prec + rec)
            if F1 > bestF1:
                bestF1 = F1
                bestEpsilon = epsilon
        except ZeroDivisionError:
            continue          
       
    return bestF1, bestEpsilon

In [153]:
F1, epsilon = selectThreshHold(y_test, pval)
print('Epsilon and F1 are:',epsilon, F1)

Epsilon and F1 are: 0.00746264644192935 0.6150442477876107


In [154]:
outl = (p < epsilon)

In [155]:
def findIndices(binVec):
    l = []
    for i in range(len(binVec)):
        if binVec[i] == 1:
            l.append(i)
    return l

In [156]:
l=findIndices(outl)
count_outliers = len(l)
print('\n\nNumber of outliers:', count_outliers)
print('\n',l)



Number of outliers: 713

 [1, 43, 75, 90, 107, 162, 178, 197, 199, 203, 211, 222, 223, 235, 240, 250, 268, 280, 291, 293, 312, 320, 350, 359, 368, 381, 386, 392, 465, 481, 485, 503, 508, 517, 523, 537, 554, 570, 574, 604, 625, 635, 661, 688, 690, 710, 716, 720, 729, 730, 734, 740, 741, 791, 796, 807, 809, 850, 905, 915, 964, 1001, 1015, 1034, 1040, 1053, 1113, 1121, 1135, 1139, 1178, 1210, 1240, 1242, 1251, 1270, 1290, 1294, 1352, 1361, 1363, 1370, 1407, 1444, 1475, 1506, 1512, 1542, 1559, 1572, 1601, 1609, 1633, 1648, 1657, 1688, 1690, 1691, 1699, 1702, 1710, 1713, 1745, 1755, 1762, 1783, 1788, 1789, 1790, 1793, 1804, 1822, 1842, 1884, 1886, 1898, 1922, 1951, 1975, 1993, 2031, 2032, 2053, 2057, 2068, 2100, 2108, 2127, 2157, 2161, 2168, 2185, 2219, 2232, 2250, 2273, 2315, 2320, 2334, 2347, 2365, 2386, 2431, 2445, 2459, 2465, 2472, 2475, 2496, 2511, 2512, 2544, 2585, 2589, 2594, 2612, 2617, 2646, 2658, 2678, 2708, 2717, 2721, 2729, 2736, 2750, 2784, 2790, 2794, 2825, 2832, 2844, 2847,

In [157]:
b=df[keep_col]
#b.head()
for i in l:
    print(b.loc[[i]])
    

   pressure
1    1023.3
1    1021.5
    pressure
43    1012.8
43    1012.1
    pressure
75    1023.1
75    1021.0
    pressure
90    1013.9
90    1012.9
     pressure
107    1012.2
107    1010.1
     pressure
162     995.2
162     994.2
     pressure
178     995.6
178     996.4
     pressure
197    1003.9
197     998.0
     pressure
199    1007.0
199     999.7
     pressure
203    1011.6
203    1004.0
     pressure
211    1016.4
211    1010.1
     pressure
222    1023.5
222    1018.4
     pressure
223    1023.7
223    1018.7
     pressure
235    1023.5
235    1020.4
     pressure
240    1024.8
240    1022.3
     pressure
250    1024.0
250    1023.2
     pressure
268    1005.6
268    1008.9
     pressure
280    1008.0
280    1002.8
     pressure
291    1014.0
291    1008.3
     pressure
293    1015.4
293    1009.2
     pressure
312    1025.4
312    1023.2
     pressure
320    1027.6
320    1025.3
     pressure
350    1023.4
350    1022.8
     pressure
359    1015.3
359    1012.4
     pr

3319    1015.1
      pressure
3335    1012.5
3335    1014.0
      pressure
3367    1018.5
3367    1013.3
         pressure
3371  1016.085292
3371  1014.200000
      pressure
3386    1016.8
3386    1017.2
      pressure
3393    1015.3
3393    1020.9
      pressure
3395    1013.5
3395    1021.4
      pressure
3402    1010.7
3402    1020.5
      pressure
3418    1009.5
3418    1016.8
      pressure
3424    1009.2
3424    1011.8
      pressure
3427    1010.1
3427    1010.3
      pressure
3443    1013.0
3443    1008.3
      pressure
3451    1013.3
3451    1008.0
      pressure
3469    1011.2
3469    1013.5
      pressure
3470    1011.2
3470    1012.9
      pressure
3479    1013.5
3479    1013.6
      pressure
3492    1014.3
3492    1012.5
      pressure
3512    1017.6
3512    1014.9
      pressure
3531    1019.1
3531    1014.5
      pressure
3576    1018.8
3576    1018.1
      pressure
3602    1019.9
3602    1019.0
      pressure
3620    1013.5
3620    1020.5
      pressure
3630    1015.2
3

7345    1015.5
      pressure
7356    1030.3
7356    1011.4
      pressure
7363    1032.4
7363    1008.7
      pressure
7395    1030.0
7395    1026.2
      pressure
7414    1026.9
7414    1025.2
      pressure
7416    1026.2
7416    1023.6
      pressure
7419    1026.7
7419    1023.4
      pressure
7437    1024.8
7437    1025.0
      pressure
7461    1016.3
7461    1037.6
      pressure
7481    1007.2
7481    1036.1
      pressure
7515    1024.1
7515    1027.0
      pressure
7527    1022.1
7527    1029.0
      pressure
7540    1023.5
7540    1028.8
      pressure
7580    1012.2
7580    1019.2
      pressure
7704    1026.3
7704    1010.0
      pressure
7709    1025.9
7709    1006.5
      pressure
7717    1023.7
7717    1014.2
      pressure
7719    1023.8
7719    1016.8
      pressure
7732    1022.5
7732    1018.3
      pressure
7734    1024.5
7734    1017.1
      pressure
7763    1022.8
7763    1024.0
      pressure
7769    1018.6
7769    1028.8
      pressure
7777    1008.3
7777    10

KeyError: "None of [Int64Index([8726], dtype='int64')] are in the [index]"